In [ ]:
from itertools import product

class Reaction:

    def __init__(self, mean: int, std: int) -> None:
        self.mean = mean
        self.std = std

class Player:

    def __init__(self, accuracy: float, reaction: Reaction) -> None:
        self.accuracy = accuracy
        self.reaction = reaction

    def __matmul__(self, other: "Player") -> "Player":
        self_speed = np.random.normal(self.reaction.mean, self.reaction.std)
        other_speed = np.random.normal(other.reaction.mean, other.reaction.std)

        self_hits = np.random.rand() < self.accuracy
        other_hits = np.random.rand() < other.accuracy
        if self_speed < other_speed:
            if self_hits:
                return 1
        elif other_hits:
            return -1
        return 0

accuracies = np.linspace(0.5, 1.0, 1000)
reaction_times = np.arange(150, 301, 10)

players = []

for _ in range(10_000):
    accuracy = np.random.normal(loc=0.75, scale=0.08)
    accuracy = max(0.0, min(1.0, accuracy))
    
    reaction_mean = int(max(150, 300 - (accuracy * 100)))
    reaction_std = int(min(30, -5 + (accuracy*35)))
    player = Player(accuracy=accuracy, reaction=Reaction(reaction_mean, reaction_std))
    players.append(player)

In [ ]:
results = []

for i in range(1_000):
    player1, player2 = np.random.choice(players, size=2)
    
    outcome = player1 @ player2
    
    match_result = {
        'Player1_Accuracy': player1.accuracy,
        'Player1_ReactionMean': player1.reaction.mean,
        'Player1_ReactionStd': player1.reaction.std,
        'Player2_Accuracy': player2.accuracy,
        'Player2_ReactionMean': player2.reaction.mean,
        'Player2_ReactionStd': player2.reaction.std,
        'Outcome': outcome
    }
    
    results.append(match_result)

df = pd.DataFrame(results)

In [ ]:
wins_player1 = df[df['Outcome'] == 1].groupby('Player1_Accuracy').size()
wins_player2 = df[df['Outcome'] == -1].groupby('Player2_Accuracy').size()

matches_player1 = df.groupby('Player1_Accuracy').size()
matches_player2 = df.groupby('Player2_Accuracy').size()

total_wins = wins_player1.add(wins_player2, fill_value=0)
total_matches = matches_player1.add(matches_player2, fill_value=0)
win_rate = (total_wins / total_matches).sort_values(ascending=False)

best_accuracy = win_rate.idxmax()
print(f"The best accuracy level is {best_accuracy:.2f} with a win rate of {win_rate[best_accuracy]:.2%}")

np.mean(win_rate.dropna()[win_rate.dropna() == 1].index)

In [ ]:
# Group by both accuracy and reaction time to analyze the trade-off
win_rates_by_acc_reaction = df.groupby(['Player1_Accuracy', 'Player1_ReactionMean'])['Outcome'].apply(
    lambda x: (x == 1).mean()
).unstack()

win_rates_by_acc_reaction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to simulate one round of player shootout
def simulate_shootout(player_a_acc, player_a_reaction, player_b_acc, player_b_reaction, num_simulations=10000):
    player_a_wins = 0
    player_b_wins = 0

    # Monte Carlo simulation
    for _ in range(num_simulations):
        # Draw random reaction times based on normal distributions
        player_a_reaction_time = np.random.normal(player_a_reaction['mean'], player_a_reaction['std'])
        player_b_reaction_time = np.random.normal(player_b_reaction['mean'], player_b_reaction['std'])

        # Simulate accuracy (1 = hit, 0 = miss)
        player_a_hits = np.random.rand() < player_a_acc
        player_b_hits = np.random.rand() < player_b_acc

        # Determine who fires first and who hits
        if player_a_reaction_time < player_b_reaction_time:
            if player_a_hits:
                player_a_wins += 1
        else:
            if player_b_hits:
                player_b_wins += 1

    return player_a_wins, player_b_wins

# Define accuracy and reaction time stats for both players
player_a_accuracy = 0.75  # 75% accuracy
player_b_accuracy = 0.95  # 95% accuracy

# Reaction times in milliseconds (mean and standard deviation for normal distribution)
player_a_reaction = {'mean': 220, 'std': 10}  # 220ms reaction time with 10ms standard deviation
player_b_reaction = {'mean': 240, 'std': 10}  # 240ms reaction time with 10ms standard deviation

# Run the Monte Carlo simulation
num_simulations = 10000
player_a_wins, player_b_wins = simulate_shootout(
    player_a_accuracy, player_a_reaction, 
    player_b_accuracy, player_b_reaction, 
    num_simulations
)

# Calculate win percentages
player_a_win_percentage = (player_a_wins / num_simulations) * 100
player_b_win_percentage = (player_b_wins / num_simulations) * 100

# Display results
print(f"Player A wins: {player_a_wins}/{num_simulations} ({player_a_win_percentage:.2f}%)")
print(f"Player B wins: {player_b_wins}/{num_simulations} ({player_b_win_percentage:.2f}%)")

# Optional: Plot the results
labels = ['Player A Wins', 'Player B Wins']
sizes = [player_a_wins, player_b_wins]
colors = ['lightblue', 'lightcoral']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Simulation Results: Win Distribution')
plt.show()